# __Step 4.4c: Fit trend line__

Goals here:
- Better detect the trends for topics

10/11/22:
- Think of this as a [forcasting problem](https://www.cienciadedatos.net/documentos/py27-time-series-forecasting-python-scikitlearn.html).
- Update `sklearn` environment but run into problem:
 - `libstdc++.so.6: version 'GLIBCXX_3.4.29' not found`
 - [Fix](https://github.com/BVLC/caffe/issues/4953): `conda install libgcc`

10/10/22: 
- The topic over time heatmap is ordered in a unstatisfactory way. While I can tell if a topic is declining or rising, it is rather subjective. Looking into [non-linear regression models for time-series](https://otexts.com/fpp2/nonlinear-regression.html), I can get a better picture by fitting trend lines.
- From [this post](https://stackoverflow.com/questions/51321100/python-natural-smoothing-splines) found the following packages:
  - https://github.com/espdev/csaps
  - https://github.com/madrury/basis-expansions
- Here is another way using [pycaret](https://towardsdatascience.com/time-series-forecasting-with-pycaret-regression-module-237b703a0c63).

## ___Set up___

### Existing environment

```
conda activate sklearn
conda update --all
pip install skforecast
```

### Module import

In [1]:
# This following is done because:
# ImportError: cannot import name '_centered' from 'scipy.signal.signaltools
#https://stackoverflow.com/questions/71106940/cannot-import-name-centered-from-scipy-signal-signaltools 

import  scipy.signal.signaltools

def _centered(arr, newsize):
    # Return the center newsize portion of the array.
    newsize = np.asarray(newsize)
    currsize = np.array(arr.shape)
    startind = (currsize - newsize) // 2
    endind = startind + newsize
    myslice = [slice(startind[k], endind[k]) for k in range(len(endind))]
    return arr[tuple(myslice)]

scipy.signal.signaltools._centered = _centered

In [2]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import statsmodels.api as sm
from pathlib import Path
from math import isnan
from scipy.interpolate import UnivariateSpline, CubicSpline
from datetime import datetime
from tqdm import tqdm

# for recursive autoregressive forcasting
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from scipy.interpolate import interp1d
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from skforecast.utils import save_forecaster
from skforecast.utils import load_forecaster

# For ARIMA
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_predict
from sklearn.metrics import mean_squared_error
import warnings
from math import sqrt
from itertools import product
import pmdarima as pm

/home/shius/miniconda3/envs/timeseries/lib/python3.10/site-packages/statsmodels/compat/pandas.py:61: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex
/home/shius/miniconda3/envs/timeseries/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Key variables

In [3]:
# Reproducibility
seed = 20220609

# Setting working directory
proj_dir   = Path.home() / "projects/plant_sci_hist"
work_dir   = proj_dir / "4_topic_model/4_4_over_time"
work_dir.mkdir(parents=True, exist_ok=True)

# modified topic names
dir43            = proj_dir / "4_topic_model/4_3_model_analysis"
toc_mod_name_file= dir43 / 'fig4_3_topic_heatmap_seaborn_order_condensed.txt'

# Topic freuqency for each timestamp
toc_freq_file    = work_dir / 'table4_4_topics_over_time_df_no_global_tune.tsv'

# So PDF is saved in a format properly
mpl.rcParams['pdf.fonttype'] = 42
plt.rcParams["font.family"] = "sans-serif"

## ___Load data___

### Load toc_mod_names

In [4]:
#https://www.adamsmith.haus/python/answers/how-to-set-column-names-when-importing-a-csv-into-a-pandas-dataframe-in-python
header_list   = ["Topic", "Name"]
toc_mod_names = pd.read_csv(toc_mod_name_file, sep='\t', names=header_list)
toc_mod_names.head()

,Topic,Name
0,-1,cell | expression | gene | protein
1,13,mirna | rnas | micrornas | target | lncrnas
2,21,circadian clock | rhythms | flowering | arabid...
3,0,allergen | pollen | ige | allergenic
4,1,medium | callus | regeneration | culture | som...


### Load and process top_freq

In [5]:
#https://stackoverflow.com/questions/28200404/pandas-read-table-use-first-column-as-index
top_freq      = pd.read_csv(toc_freq_file, index_col=0, sep='\t')
top_freq.head()

,Topic,Words,Frequency,Timestamp
0,-1,"cells, growth, acid, activity, tissue",981,250750799.0
1,0,"timothy, timothy pollen, antigen, ragweed, all...",8,250750799.0
2,1,"callus, medium, kinetin, culture, protoplasts",85,250750799.0
3,2,"berberinium, viscometric titrations, flow pola...",2,250750799.0
4,3,"amiben, atrazine, gsatrazine, atrazine metabol...",4,250750799.0


In [6]:
# Turn the dataframe into a matrix with rows as timestamps, column as topics,
# and values as frequency
top_freq_dict = {} # {timestamp:{topic:freq}}
for idx in top_freq.index:
  row  = top_freq.loc[idx]
  ts   = row["Timestamp"]
  toc  = row["Topic"]
  freq = row["Frequency"]
  if ts not in top_freq_dict:
    top_freq_dict[ts] = {toc:freq}
  elif toc not in top_freq_dict[ts]:
    top_freq_dict[ts][toc] = freq
  else:
    print("ERR: redundant", topic,freq)

In [7]:
# Fill in 0s
#ts_unique = top_freq.Timestamp.unique()
#topics    = top_freq.Topic.unique()

# Go through each ts_unique and topics
#for ts in ts_unique:
#  if ts not in top_freq_dict:
#    top_freq_dict[ts] = {}
#  for toc in topics:
#    if toc not in top_freq_dict[ts]:
#      top_freq_dict[ts][toc] = 0

# Realize that I can use DataFrame.fillna after dataframe is created
top_freq_df = pd.DataFrame.from_dict(top_freq_dict)
top_freq_df.fillna(0, inplace=True)
top_freq_df.sort_index(axis=0, inplace=True)  # sort rows
top_freq_df.sort_index(axis=1, inplace=True)  # sort columns
top_freq_df.index.name = "Topic"
top_freq_df.head()

,2.507508e+08,4.258800e+08,5.468400e+08,6.389460e+08,7.126416e+08,7.783920e+08,8.361936e+08,8.975376e+08,9.497268e+08,9.869616e+08,...,1.521864e+09,1.532750e+09,1.543986e+09,1.554178e+09,1.564027e+09,1.574053e+09,1.583557e+09,1.592712e+09,1.601438e+09,1.609477e+09
Topic,,,,,,,,,,,,,,,,,,,,,
-1,981.0,899.0,949.0,971.0,997.0,910.0,1013.0,1032.0,1139.0,1074,...,1050,960.0,1013.0,927,934.0,977.0,996.0,957,980.0,896.0
0,8.0,31.0,31.0,35.0,28.0,47.0,44.0,67.0,58.0,30,...,4,7.0,6.0,5,6.0,3.0,4.0,7,4.0,6.0
1,85.0,87.0,211.0,222.0,212.0,159.0,144.0,153.0,58.0,94,...,30,22.0,19.0,21,22.0,18.0,27.0,26,29.0,24.0
2,2.0,1.0,2.0,4.0,0.0,2.0,6.0,3.0,2.0,6,...,37,32.0,45.0,53,31.0,40.0,50.0,48,36.0,33.0
3,4.0,2.0,2.0,8.0,26.0,11.0,5.0,6.0,6.0,6,...,26,17.0,11.0,24,21.0,22.0,23.0,22,17.0,18.0


In [8]:
top_freq_df.to_csv(work_dir / 'table4_4c_topic_frequency_per_timestamp.txt')

## ___Forcasting___ 

### Set up

In [9]:
# output folder
dir_forcast = work_dir / "_forcast"
dir_forcast.mkdir(parents=True, exist_ok=True)

In [10]:
ts_unique    = top_freq_df.columns.to_list()
topics       = top_freq_df.index.to_list()

In [11]:
def get_asfreq_series(row_series):
  '''This is to get a pandas series that with day info set to 1, and fill in
     missing months with NaN.
    Args:
      row_series (Series): timestamps as indices and doc frequency as values
    Return
      y_df (DataFrame): with dates as indicies and a column of y values
  '''
  x  = row_series.index.to_list()
  y  = row_series.values.tolist()

  # Turn timestamps to dates
  x_dates = [datetime.fromtimestamp(ts+1) for ts in x]
  # Change all dates to 1st day of the month. This is done otherwise anything
  # not on the 1st day of the month will be removed when df.asfreq is applied.
  x_dates = [datetime.strptime(d.strftime("%Y:%m"), "%Y:%m") for d in x_dates]

  # Create a pandas series with y using dates as indices
  y_df = pd.DataFrame({"date":x_dates, "y":y})
  y_df = y_df.set_index('date')

  # Set frequency to monthly and fill in missing data as NaN
  y_df = y_df.asfreq('MS') 
  y_df = y_df.sort_index() # make sure the date is sorted

  return y_df  

### ForecasterAutoreg

The results are linear, does not look right. This approach also requires missing data to be imputed. There are simply too many missing values at the month level.

After working on ARIMA, the results pretty much the same, both crappy.

In [12]:
def fit_ForecasterAutoreg(toc, row_series, steps_test, steps_future, dir_forcast):
  '''Fit recursive autoregressors with data for a topic
  Args:
    toc (int): topic
    row_series (Series): a pandas series with dates as indices and frequency as 
      values for a topic that derives from a row in the topic-timestamp 
      dataframe
    steps_test (int): number of data witheld for testing
    steps_future (int): number of months to predict into the future
    dir_forcast (Path): directory for forcast results
  Returns:
    predictions_future (Series): the future prediction values
  Output:
    plots (pdf): original data + predictions
    values (txt): original data + predictions
  '''
  y_df = get_asfreq_series(row_series)

  # impute missing data
  y_df['y_spline'] = y_df['y'].interpolate(option='spline')

  # Set last few months as test set
  data_train = y_df[:-steps_test]
  data_test  = y_df[-steps_test:]

  # train ForcasterAutoreg
  forecaster_ar = ForecasterAutoreg(
                  regressor = RandomForestRegressor(random_state=seed),
                  lags      = 6)

  # Lags used as predictors
  lags_grid = [10, 20]

  # Regressor's hyperparameters
  param_grid = {'n_estimators': [50, 100, 200], 
                'max_depth': [3, 5, 10]}

  results_grid = grid_search_forecaster(
                          forecaster         = forecaster_ar,
                          y                  = data_train['y_spline'],
                          param_grid         = param_grid,
                          lags_grid          = lags_grid,
                          steps              = steps_test,
                          refit              = True,
                          metric             = 'mean_squared_error',
                          initial_train_size = int(len(data_train)*0.5),
                          fixed_train_size   = False,
                          return_best        = True,
                          verbose            = False)

  # get best parameters
  num_lags     = len(results_grid.iloc[0]['lags'])
  max_depth    = results_grid.iloc[0]['max_depth']
  n_estimators = results_grid.iloc[0]['n_estimators'] 
  
  # generate final model
  regressor_rf = RandomForestRegressor(max_depth=5, n_estimators=100, 
                                     random_state=seed)
  forecaster_final = ForecasterAutoreg(regressor = regressor_rf,
                                      lags      = 20)
  forecaster_final.fit(y=data_train['y_spline'])

  # generate predictions
  predictions_test            = forecaster_final.predict(steps=steps_test)
  predictions_test_and_future = forecaster_final.predict(steps=steps_test + \
                                                               steps_future)
  predictions_future = predictions_test_and_future[steps_test:]

  # plot orignal, imputed, and prediction values
  fig, ax=plt.subplots(figsize=(9, 4))
  data_train['y_spline'].plot(ax=ax, label='train')
  data_test['y_spline'].plot(ax=ax, label='test')
  y_df['y'].plot(ax=ax, style='o', ms=5, label='original data')
  predictions_future.plot(ax=ax, label='predictions:test + future')
  fig.savefig(dir_forcast / f'figure4_4c_forcast_topic_{toc}_autoreg.pdf')

  return predictions_future


In [ ]:
# Did not let it finish
'''
steps_test   = 24
steps_future = 120
c = 0
for toc in tqdm(topics):
  row_series = top_freq_df.loc[toc]
  predictions_future = fit_ForecasterAutoreg(toc, row_series, steps_test, 
                                             steps_future, dir_forcast)
'''

### ARIMA

- Functions from [ML Mastery](https://machinelearningmastery.com/grid-search-arima-hyperparameters-with-python/)
- Discussion on [ARIMA with missing values](https://github.com/statsmodels/statsmodels/issues/6596)
- There is also pmdarima that does auto ARIMA in [this post section 12](https://www.machinelearningplus.com/time-series/arima-model-time-series-forecasting-python/). But this does not deal with missing values. So still need to impute.

#### Functions

In [ ]:
def run_auto_arima(series):
  model = pm.auto_arima(series, 
                        start_p=0, start_q=0,
                        test='adf',       # use adftest to find optimal 'd'
                        max_p=3, max_q=3, # maximum p and q
                        m=1,              # frequency of series
                        d=None,           # let model determine 'd'
                        seasonal=False,   # No Seasonality
                        start_P=0, 
                        D=0, 
                        error_action='ignore',  
                        suppress_warnings=True, 
                        stepwise=True,
                        trace=False)

  best_param = model.arima_res_.specification['order']
  
  return model, best_param

In [ ]:
# NOT USED: slow
# Another implementation doing grid search
# change function name
def grid_search_arima(dataset, p_values, d_values, q_values):
	dataset = dataset.astype('float32')
	best_score, best_cfg = float("inf"), None

	# 10/12/22: Modify the original function to get a list of combinations so
	#   tqdm can be used.
	combo = list(product(p_values, d_values, q_values))

	for order in tqdm(combo):
		try:
			mse = evaluate_arima_model(dataset, order)
			if mse < best_score:
				best_score, best_cfg = mse, order
			#print('ARIMA%s MSE=%.3f' % (order,mse))
		except:
			continue
	print('Best ARIMA%s MSE=%.3f' % (best_cfg, best_score))
	
	# added
	return best_score, best_cfg

In [ ]:
# NOT USED: part of the function above
# evaluate an ARIMA model for a given order (p,d,q)
def evaluate_arima_model(X, arima_order):
	# prepare training dataset
	train_size = int(len(X) * 0.66)
	train, test = X[0:train_size], X[train_size:]
	history = [x for x in train]
	# make predictions
	predictions = list()
	for t in range(len(test)):
		model = ARIMA(history, order=arima_order)
		model_fit = model.fit()
		yhat = model_fit.forecast()[0]
		predictions.append(yhat)
		history.append(test[t])
	# calculate out of sample error
	rmse = sqrt(mean_squared_error(test, predictions))
	return rmse

#### Fit ARIMA for each topic

In [ ]:
c = 0
#p_values = [0, 1, 2, 4, 6, 8, 10]
#d_values = range(0, 3)
#q_values = range(0, 3)

for toc in tqdm(topics):
  # Get the timestamp-freq series for a topic
  toc_series   = top_freq_df.loc[toc]
  # Get a dataframe where the rows are in months, months without values have NaN
  toc_y_df     = get_asfreq_series(toc_series)
  # Impute NaN with spline
  toc_y_df_imp = toc_y_df['y'].interpolate(option='spline')

  # Run auto ARIMA and get best param
  model_aa, best_param = run_auto_arima(toc_y_df_imp)
  #_, best_param = grid_search_arima(toc_y_df_imp, p_values, d_values, q_values)
  
  # Run ARIMA again with best param
  model_arima = ARIMA(toc_y_df_imp, order=best_param)
  model_fit   = model_arima.fit()

  # Plottting
  fig, ax = plt.subplots()
  ax = toc_y_df_imp.iloc[0:].plot(ax=ax)
  plot_predict(model_fit, '2020-01-01', '2024-12-01', ax=ax)
  toc_y_df['y'].plot(ax=ax, style='o', ms=5, label='original data')
  file_name = dir_forcast / f'figure4_4c_forcast_topic_{toc}_arima.pdf'
  fig.savefig(file_name)

## ___Curve fitting: LOWESS___

### Set output dir

In [ ]:
# output folder
dir_lowess = work_dir / "_lowess"
dir_lowess.mkdir(parents=True, exist_ok=True)

### Function

In [ ]:
def get_lowess(toc, x, y, frac, it=3, plot=0):

  lowess = sm.nonparametric.lowess
  lowess_fit = lowess(y, x, frac=frac, it=it) 
  lowess_x   = list(zip(*lowess_fit))[0]
  lowess_y   = list(zip(*lowess_fit))[1]

  # create a function using the interp1d method
  f     = interp1d(lowess_x, lowess_y, bounds_error=False)

  #x_line = [i/10. for i in range(400)]
  # define a sequence of inputs between the smallest and largest known inputs
  x_line = np.arange(min(x), max(x), (max(x)-min(x))/100)
  #x_date = [datetime.fromtimestamp(ts) for ts in lowess_x]
  y_line = f(x_line)

  # Create series
  lowess_ser = pd.Series(lowess_y, index=lowess_x, name=toc)

  if plot:
    # get 2 decimal points for mse
    mse = "{:.2f}".format(mean_squared_error(y, lowess_y))
    plt.title(f"topic {toc}, MSE={mse}")
    plt.plot(x, y, 'o')
    #plt.plot(lowess_x, lowess_y, '*')
    plt.plot(lowess_x, lowess_y, '*')
    #plt.plot(x_new, y_new, '-')
    plt.plot(x_line, y_line, '-')
    plt.ylim([0, max([max(y), max(lowess_y)])])
    plt.savefig(dir_lowess / f'figure4_4c_topic_{toc}_lowess.pdf')
    plt.close()
    
  return lowess_ser 

### Iterate through topics

In [ ]:
topics

In [ ]:
max_y_idxs      = [] # indices (x, timestamps) with max y
lowess_ser_list = [] # list of lowess-fitted series for different topics
for toc in topics:
  # series for the topic, timestamps as indices, frequencies as row
  toc_series = top_freq_df.loc[toc]
  x          = toc_series.index  # timestamps
  y          = toc_series.values # frequencies

  # series after lowess smoothing
  lowess_ser = get_lowess(toc, x, y, 1/10, 3, plot=1)

  max_y_idx  = lowess_ser.idxmax()
  max_y_idxs.append(max_y_idx)
  lowess_ser_list.append(lowess_ser)
  print(toc, max_y_idx)

### Output fitted values and topic order

In [ ]:
# for topic order
max_y_idxs_ser = pd.Series(max_y_idxs, index=topics, name='max_y_timestamp')
max_y_idxs_ser.to_csv(dir_lowess / "table4_4c_timestamps_with_max_y.txt")
max_y_idxs_ser

In [ ]:
#https://sparkbyexamples.com/pandas/pandas-create-dataframe-from-multiple-series/
lowess_df = pd.concat(lowess_ser_list, axis=1).transpose()

#https://www.geeksforgeeks.org/how-to-sort-a-pandas-dataframe-based-on-column-names-or-row-index/
lowess_df.sort_index(inplace=True, axis=1)

lowess_df.head()

In [ ]:
lowess_df.columns

In [ ]:
lowess_df.to_csv(dir_lowess / "table4_4c_toc_timestamps_lowess_vals.txt")

## ___Code testing___

### Testing data

In [ ]:
ts_unique = top_freq_df.columns.to_list()

data = top_freq_df.iloc[1]
x  = data.index.to_list()
y  = data.values.tolist()
x[:5], y[:5]

### Scipy natural cubic smoothing spline

In [ ]:
# Same cubic spline (k=3) as before
np.random.seed(seed)
spl    = UnivariateSpline(x, y, k=3, ext=0)
xs     = np.linspace(x[0], x[-1], 150) 
ys_spl = spl(xs)
plt.plot(x, y, 'o', xs, ys_spl, '-')
plt.show()

### Scipy cubic spline

In [ ]:
cs    = CubicSpline(x, y, bc_type='natural')
ys_cs = cs(xs)
plt.plot(x, y, 'o', xs, ys_cs, '-')
plt.show()

### Recursive autoregressive forecasting

https://www.cienciadedatos.net/documentos/py27-time-series-forecasting-python-scikitlearn.html

#### Create dataframe

In [ ]:
len(y)

In [ ]:
# Turn timestamps to dates
x_dates = [datetime.fromtimestamp(ts+1) for ts in x]

# Change all dates to 1st day of the month. This is done otherwise anything
# not on the 1st day of the month will be removed when df.asfreq is applied.
x_dates = [datetime.strptime(d.strftime("%Y:%m"), "%Y:%m") for d in x_dates]
x_dates

In [ ]:
# Create a pandas series with y using dates as indices
y_df = pd.DataFrame({"date":x_dates, "y":y})
y_df = y_df.set_index('date')

# Set frequency to monthly and fill in missing data as NaN
y_df = y_df.asfreq('MS') 
y_df = y_df.sort_index() # make sure the date is sorted
print(f'#rows: {y_df.shape[0]}, #missing_val:{y_df.isnull().any(axis=1).sum()}')

In [ ]:
# impute missing data
y_df['y_spline'] = y_df['y'].interpolate(option='spline')
print('#rows:', y_df.shape[0], 
      '#missing_val:',y_df['y_spline'].isnull().any().sum())
plt.plot(y_df['y_spline'], 'yo')
plt.plot(y_df['y'], 'r+')
plt.legend(["y_spline", "y"])
plt.show()

In [ ]:
# Verify that a temporary index is complete
(y_df.index == pd.date_range(start=y_df.index.min(),
                             end=y_df.index.max(),
                             freq=y_df.index.freq)).all()

In [ ]:
# Set last few months as test set
steps = 36
data_train = y_df[:-steps]
data_test  = y_df[-steps:]

print(f"Train dates:{data_train.index.min()}-{data_train.index.max()} (n={len(data_train)})")
print(f"Test dates :{data_test.index.min()}-{data_test.index.max()} (n={len(data_test)})")

#### ForcasterAutoReg

In [ ]:
forecaster_ar = ForecasterAutoreg(
                regressor = RandomForestRegressor(random_state=seed),
                lags      = 6)

# Lags used as predictors
lags_grid = [5, 10, 20]

# Regressor's hyperparameters
param_grid = {'n_estimators': [100, 500], 'max_depth': [3, 5, 10]}

results_grid = grid_search_forecaster(
                        forecaster         = forecaster_ar,
                        y                  = data_train['y_spline'],
                        param_grid         = param_grid,
                        lags_grid          = lags_grid,
                        steps              = steps,
                        refit              = True,
                        metric             = 'mean_squared_error',
                        initial_train_size = int(len(data_train)*0.5),
                        fixed_train_size   = False,
                        return_best        = True,
                        verbose            = False)


In [ ]:
# best result with smallest mse
results_grid.iloc[0]

In [ ]:
print("num_lags=",len(results_grid.iloc[0]['lags']))

#### Final model

In [ ]:
regressor_rf = RandomForestRegressor(max_depth=5, n_estimators=100, 
                                     random_state=seed)
forecaster_final = ForecasterAutoreg(regressor = regressor_rf,
                                     lags      = 20)
forecaster_final.fit(y=data_train['y_spline'])
predictions_test = forecaster_final.predict(steps=steps)

In [ ]:
type(predictions_test)

In [ ]:
# the next 10 years
steps_future = 120
predictions_future = forecaster_final.predict(steps=steps+steps_future)

In [ ]:
fig, ax=plt.subplots(figsize=(9, 4))
data_train['y_spline'].plot(ax=ax, label='train')
data_test['y_spline'].plot(ax=ax, label='test')
y_df['y'].plot(ax=ax, style='o', ms=5, label='original data')
predictions_future.plot(ax=ax, label='predictions:test + future')
ax.legend()

In [ ]:
# Test error
error_mse = mean_squared_error(y_true = data_test['y_spline'],
                               y_pred = predictions_test)
print(f"Test error (mse): {error_mse}")

In [ ]:
predictions_future[steps:].shape

### ARIMA plot

https://stackoverflow.com/questions/73112516/arimaresults-object-has-no-attribute-plot-predict-error

In [ ]:
import statsmodels.api as sm

dta = sm.datasets.sunspots.load_pandas().data[['SUNACTIVITY']]

# freq alias: 'A': year end frequency
#https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-offset-aliases
dta.index = pd.date_range(start='1700', end='2009', freq='A')
dta.plot()

In [ ]:

res = ARIMA(dta, order=(0,2,0)).fit()
fig, ax = plt.subplots()
ax = dta.loc['1950':].plot(ax=ax)
plot_predict(res, '1990', '2012', ax=ax)
plt.show()

### Testing ARIMA

In [ ]:
test_series = top_freq_df.loc[1]
test_y_df   = get_asfreq_series(test_series)
test_y_df['y_spline'] = test_y_df['y'].interpolate(option='spline')
test_y_df.head()

In [ ]:
type(test_y_df.y_spline)

In [ ]:
model_aa = run_auto_arima(test_y_df.y_spline)

In [ ]:
help(model_aa)

In [ ]:
help(model_aa.arima_res_)

In [ ]:
best_param = model_aa.arima_res_.specification['order']
best_param

In [ ]:
model_arima = ARIMA(test_y_df['y_spline'], order=best_param)
model_fit   = model_arima.fit()
print(model_fit.summary())

In [ ]:
test_y_df.index

In [ ]:
#https://stackoverflow.com/questions/73112516/arimaresults-object-has-no-attribute-plot-predict-error
fig, ax = plt.subplots()
ax = test_y_df['y_spline'].iloc[0:].plot(ax=ax)
plot_predict(model_fit, '2018-01-01', '2024-12-01', ax=ax)
test_y_df['y'].plot(ax=ax, style='o', ms=5, label='original data')
plt.show()

### Fitting directly with missing data

In [ ]:
test_series

In [ ]:
  x  = test_series.index.to_list()
  y  = test_series.values.tolist()

  # Turn timestamps to dates
  x_dates  = [datetime.fromtimestamp(ts+1) for ts in x]
  # Change all dates to 1st day of the month. This is done otherwise anything
  # not on the 1st day of the month will be removed when df.asfreq is applied.
  x_dates2 = [datetime.strptime(d.strftime("%Y:%m"), "%Y:%m") for d in x_dates]

  # Create a pandas series with y using dates as indices
  y_df_ori = pd.DataFrame({"date":x_dates, "y":y})
  y_df_mod = pd.DataFrame({"date":x_dates2, "y":y})
  y_df_ori = y_df_ori.set_index('date')
  y_df_mod = y_df_mod.set_index('date')
  y_df

In [ ]:
y_df_ori.index = pd.DatetimeIndex(y_df_ori.index).to_period('M')
y_df_ori.head()

In [ ]:
# Run ARIMA with original series with timestamps
model_arima = ARIMA(y_df_ori['y'], order=(1,1,0))
model_fit   = model_arima.fit()

In [ ]:
fig, ax = plt.subplots()
ax = y_df_ori['y'].iloc[0:].plot(ax=ax)
plot_predict(model_fit, '2018-01', '2024-12', ax=ax)
y_df_ori['y'].plot(ax=ax, style='o', ms=5, label='original data')
plt.show()

### Test LOWESS curve fitting

In [ ]:
toc_series = top_freq_df.loc[topics[0]]
x = toc_series.index
y = toc_series.values
x, y

In [ ]:
def get_lowess(x, y, frac, it=3, plot=0):

  lowess = sm.nonparametric.lowess
  lowess_fit = lowess(y, x, frac=frac, it=it) 
  lowess_x   = list(zip(*lowess_fit))[0]
  lowess_y   = list(zip(*lowess_fit))[1]

  print("MSE=", mean_squared_error(y, lowess_y))

  # create a function using the interp1d method
  f     = interp1d(lowess_x, lowess_y, bounds_error=False)

  #x_line = [i/10. for i in range(400)]
  # define a sequence of inputs between the smallest and largest known inputs
  x_line = np.arange(min(x), max(x), (max(x)-min(x))/100)
  #x_date = [datetime.fromtimestamp(ts) for ts in lowess_x]
  y_line = f(x_line)
  if plot:
    plt.plot(x, y, 'o')
    #plt.plot(lowess_x, lowess_y, '*')
    plt.plot(lowess_x, lowess_y, '*')
    #plt.plot(x_new, y_new, '-')
    plt.plot(x_line, y_line, '-')
    plt.ylim([0, max([max(y), max(lowess_y)])])
    plt.show()

  lowess_ser = pd.Series(lowess_y, index=lowess_x)
  max_x = lowess_ser.max()
  return lowess_ser, max_x 

In [ ]:
lowess_ser, max_x = get_lowess(x, y, 1/10, 3, plot=1)
lowess_ser, datetime.fromtimestamp(max_x)